In [17]:
import os
import cv2
import torch
import numpy as np
from PIL import Image
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [18]:
INPUT_SIZE = 256
image_directory = r'E:/Project/2024 Project/BrainTumor_Lam/datasets/'

no_tumor_images = os.listdir(image_directory + 'sort_crop_no/')
yes_tumor_images = os.listdir(image_directory + 'sort_crop_yes/')
dataset = []
labels = []


In [19]:
for image_name in no_tumor_images:
    if image_name.endswith('.jpg'):
        image = cv2.imread(image_directory + 'sort_crop_no/' + image_name)
        image = Image.fromarray(image, 'RGB')
        image = image.resize((INPUT_SIZE, INPUT_SIZE))
        dataset.append(np.array(image))
        labels.append(0)

for image_name in yes_tumor_images:
    if image_name.endswith('.jpg'):
        image = cv2.imread(image_directory + 'sort_crop_yes/' + image_name)
        image = Image.fromarray(image, 'RGB')
        image = image.resize((INPUT_SIZE, INPUT_SIZE))
        dataset.append(np.array(image))
        labels.append(1)

In [20]:
dataset = np.array(dataset)
labels = np.array(labels)

x_train, x_test, y_train, y_test = train_test_split(dataset, labels, test_size=0.2, random_state=42)

x_train = torch.tensor(x_train, dtype=torch.float32)
x_test = torch.tensor(x_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

x_train = x_train.permute(0, 3, 1, 2) / 255.0 
x_test = x_test.permute(0, 3, 1, 2) / 255.0
# chuyển tenser đang từ (batch, height, width, channel) sang (batch, channel, height, width) 

train_data = torch.utils.data.TensorDataset(x_train, y_train)
test_data = torch.utils.data.TensorDataset(x_test, y_test)
# TensorDataset nhận các tensor đầu vào và tạo thành một dataset trong đó, mỗi phần tử là một tuple chứa mẫu dữ liệu và nhãn tương ứng

train_loader = DataLoader(train_data, batch_size=16, shuffle=True)
test_loader = DataLoader(test_data, batch_size=16, shuffle=False)


In [21]:
class BrainTumorNet(nn.Module):
    def __init__(self):
        super(BrainTumorNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1) # 3 channel đầu vào, 32 channel đầu ra, padding = 1 tức là thêm 1 hàng 0 vào 2 bên để giữ nguyên kích thước
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * (INPUT_SIZE // 8) * (INPUT_SIZE // 8), 64) 
        # INPUT_SIZE // 8 vì sau 3 lớp convolution và 3 lớp pooling, kích thước ảnh giảm đi 8 lần
        # 64 là số lượng kênh vào từ lớp tích chập cuối cùng , 64 là số nút đầu ra của lớp fully connected
        self.fc2 = nn.Linear(64, 1)
        #64 đầu vào và 1 đầu ra
        self.dropout = nn.Dropout(0.3)  # Increased dropout rate to 0.3 for regularization

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = F.relu(self.conv3(x))
        x = self.pool(x)
        x = x.view(-1, 64 * (INPUT_SIZE // 8) * (INPUT_SIZE // 8))
        # view dùng để thay đổi hình dạng của tensor, -1 tức là giữ nguyên số hàng, còn lại là 64 * (INPUT_SIZE // 8) * (INPUT_SIZE // 8)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.sigmoid(self.fc2(x))
        return x

In [22]:
model = BrainTumorNet()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

Chức Năng Của optimizer.zero_grad()
Đặt Lại Gradient:
Trong PyTorch, gradient của các tham số mô hình được tính toán và lưu trữ trong đối tượng Tensor khi bạn gọi loss.backward(). Tuy nhiên, các giá trị gradient này sẽ được cộng dồn (accumulated) qua nhiều lần tính toán gradient liên tiếp nếu bạn không đặt lại chúng.
optimizer.zero_grad() được sử dụng để đặt lại tất cả các gradient của các tham số mô hình về 0 trước khi tính toán gradient cho lần huấn luyện tiếp theo.
Tránh Tích Lũy Gradient:
Nếu bạn không gọi zero_grad(), gradient của các tham số sẽ được cộng dồn qua các lần gọi loss.backward(). Điều này có thể dẫn đến các gradient lớn không mong muốn và làm sai lệch quá trình tối ưu hóa.
Đặt lại gradient về 0 đảm bảo rằng gradient của mỗi batch được tính toán độc lập với các batch khác.

In [23]:
criterion = nn.BCELoss()  #binary cross entropy loss
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Learning rate scheduler
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True, min_lr=1e-6)
# chay them epoch

# Training loop
num_epochs = 30
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device, dtype=torch.float32)
        
        optimizer.zero_grad() #đặt lại gradient về 0
        
        outputs = model(inputs).squeeze() #squeeze() giảm một chiều của tensor
        loss = criterion(outputs, labels) # tính loss
        loss.backward() # tính gradient 
        optimizer.step() # cập nhật trọng số
        
        running_loss += loss.item()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

    # Validation
    model.eval() #đặt mô hình ở chế độ evaluation
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device, dtype=torch.float32)
            outputs = model(inputs).squeeze()
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            
            predicted = (outputs > 0.5).float()
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    print(f"Validation Loss: {val_loss/len(test_loader):.4f}, Val_Accuracy: {100 * correct / total:.2f}%")
    
    # Adjust learning rate
    scheduler.step(val_loss)

save_path = 'E:/Project/2024 Project/BrainTumor_Lam/models/brain_tumor_pytorch_v1.pth'
# Save the model
torch.save(model.state_dict(), save_path)


c:\Users\ADMIN\miniconda3\envs\brain_tumor_detection\Lib\site-packages\torch\optim\lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch [1/30], Loss: 0.6848
Validation Loss: 0.6552, Val_Accuracy: 64.65%
Epoch [2/30], Loss: 0.5978
Validation Loss: 0.5251, Val_Accuracy: 78.14%
Epoch [3/30], Loss: 0.4596
Validation Loss: 0.4249, Val_Accuracy: 80.47%
Epoch [4/30], Loss: 0.3209
Validation Loss: 0.3036, Val_Accuracy: 88.60%
Epoch [5/30], Loss: 0.2211
Validation Loss: 0.3043, Val_Accuracy: 88.37%
Epoch [6/30], Loss: 0.1594
Validation Loss: 0.2619, Val_Accuracy: 91.63%
Epoch [7/30], Loss: 0.1259
Validation Loss: 0.2943, Val_Accuracy: 91.40%
Epoch [8/30], Loss: 0.0847
Validation Loss: 0.3767, Val_Accuracy: 91.16%
Epoch [9/30], Loss: 0.0893
Validation Loss: 0.3494, Val_Accuracy: 91.40%
Epoch [10/30], Loss: 0.0739
Validation Loss: 0.3484, Val_Accuracy: 92.56%
Epoch [11/30], Loss: 0.0421
Validation Loss: 0.5786, Val_Accuracy: 92.56%
Epoch [12/30], Loss: 0.0394
Validation Loss: 0.5966, Val_Accuracy: 92.79%
Epoch [13/30], Loss: 0.0319
Validation Loss: 0.6209, Val_Accuracy: 92.56%
Epoch [14/30], Loss: 0.0343
Validation Loss: 0.

In [24]:
model.eval()
y_pred = []
y_true = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        outputs = model(inputs).squeeze()
        predicted = (outputs > 0.5).cpu().numpy().astype(int)
        y_pred.extend(predicted)
        y_true.extend(labels.numpy())

f1 = f1_score(y_true, y_pred)
print(f"F1 Score: {f1}")

F1 Score: 0.9194805194805195
